## Alzheimer's Classification (4 classes of images)

The goal is to develop a simple classification model, and then test a variety of XAI techniques in order to understand what method fits best to alzheimer classification explainbility. 

In [1]:
!pip install tensorflow-addons==0.16.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 9.6 MB/s 


# Imports

In [20]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import os
from tqdm import tqdm
from PIL import Image

from sklearn.utils import shuffle
from sklearn import metrics
import matplotlib.pyplot as plt
from tensorflow.keras import Input, Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D, GlobalAveragePooling2D, SeparableConv2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import keras.backend as K

from distutils.dir_util import copy_tree, remove_tree

# Loading Dataset

In [7]:
train_data_gen = ImageDataGenerator(rescale = 1./255,
                                    validation_split = 0.25,
                                    rotation_range=10,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    fill_mode='nearest')

valid_data_gen = ImageDataGenerator(rescale = 1./255, validation_split = 0.25)

test_data_gen  = ImageDataGenerator(rescale = 1./255)

In [8]:
training_data  = train_data_gen.flow_from_directory(directory = '/content/drive/MyDrive/MSc/Data/Alzheimer_s Dataset/train',
                                                   target_size = (224,224),
                                                   class_mode = 'categorical',
                                                   subset = 'training',
                                                   batch_size = 32)

Found 3841 images belonging to 4 classes.


In [28]:
valid_data = valid_data_gen.flow_from_directory(directory = '/content/drive/MyDrive/MSc/Data/Alzheimer_s Dataset/train',
                                                  target_size = (224,224),
                                                  class_mode = 'categorical',
                                                  subset = 'validation',
                                                  batch_size = 32)

Found 1280 images belonging to 4 classes.


In [10]:
test_data = test_data_gen.flow_from_directory(directory = '/content/drive/MyDrive/MSc/Data/Alzheimer_s Dataset/test',
                                                  target_size = (224,224),
                                                  class_mode = 'categorical',
                                                  batch_size = 32)

Found 1279 images belonging to 4 classes.


# Model Building

In [21]:
def sequential_model():
    model = Sequential()
    model.add(Conv2D(16,(3,3),padding='valid',input_shape = (224,224,3),activation='relu'))
    model.add(Conv2D(32,(3,3),padding='valid',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding = 'same'))
    model.add(Conv2D(32,(3,3),padding='valid',activation='relu'))   
    model.add(Conv2D(64,(3,3),padding='valid',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding = 'same'))
    
    model.add(Conv2D(64,(3,3),padding='valid',activation='relu'))
    
    model.add(Conv2D(128,(3,3),padding='valid',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding = 'same'))
    
    model.add(Conv2D(128,(3,3),padding='valid',activation='relu'))
    
    model.add(Conv2D(128,(3,3),padding='valid',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding = 'same'))
    
    
    model.add(Flatten())
    model.add(Dense(32))
    model.add(Dense(32))
    model.add(Dense(32))
    model.add(Dense(32))
    model.add(Dense(4))
    model.add(Activation('softmax'))
    
    return model


In [22]:
model = sequential_model()
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 222, 222, 16)      448       
                                                                 
 conv2d_19 (Conv2D)          (None, 220, 220, 32)      4640      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 110, 110, 32)     0         
 2D)                                                             
                                                                 
 conv2d_20 (Conv2D)          (None, 108, 108, 32)      9248      
                                                                 
 conv2d_21 (Conv2D)          (None, 106, 106, 64)      18496     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 53, 53, 64)       0         
 2D)                                                  

In [23]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [24]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),  
      tf.keras.metrics.AUC(name='auc'),
      get_f1,
]

In [25]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 5) # when i run it for 50 epochs

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

In [26]:
model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam',), loss='categorical_crossentropy',metrics=METRICS)

In [29]:
history=model.fit(training_data,
                        validation_data=valid_data,
                        epochs = 10,
                        verbose = 1,
                        callbacks=lr_scheduler)

Epoch 1/10
121/121 [==============================] - 1059s 9s/step - loss: 37.8542 - accuracy: 0.7445 - precision: 0.4798 - recall: 0.2598 - auc: 0.7634 - get_f1: 0.2582 - val_loss: 1.0513 - val_accuracy: 0.7500 - val_precision: 0.5000 - val_recall: 0.5000 - val_auc: 0.7799 - val_get_f1: 0.5000 - lr: 0.0100
Epoch 2/10
121/121 [==============================] - 51s 417ms/step - loss: 1.0456 - accuracy: 0.7523 - precision: 0.5067 - recall: 0.3421 - auc: 0.7783 - get_f1: 0.3474 - val_loss: 1.0474 - val_accuracy: 0.7500 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.7799 - val_get_f1: 0.0000e+00 - lr: 0.0063
Epoch 3/10
121/121 [==============================] - 51s 418ms/step - loss: 1.0419 - accuracy: 0.7496 - precision: 0.4983 - recall: 0.2325 - auc: 0.7798 - get_f1: 0.2306 - val_loss: 1.0399 - val_accuracy: 0.7500 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.7799 - val_get_f1: 0.0000e+00 - lr: 0.0040
Epoch 4/10
121/121 [==========================